In [1]:
import os
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datetime import datetime

In [ ]:
# Bước 1: Đọc dữ liệu từ CSV
data = pd.read_csv(os.path.join("AI Research Dataset","AI Research Paper Dataset - Cleanned.csv"))

# Bước 2: Tiền xử lý dữ liệu
# Chuyển ngày thành timestamp
data['Published'] = pd.to_datetime(data['Published']).map(datetime.timestamp)
data['Updated'] = pd.to_datetime(data['Updated']).map(datetime.timestamp)

# Mã hóa các cột phân loại: Authors, Primary Category
encoder = OneHotEncoder()
authors_encoded = encoder.fit_transform(data[['Authors']]).toarray()
category_encoded = encoder.fit_transform(data[['Primary Category']]).toarray()

# Chuẩn bị dữ liệu văn bản (Title + Summary)
texts = data['Title'] + ' ' + data['Summary']

# Sử dụng Tokenizer để chuyển đổi văn bản thành số
tokenizer = Tokenizer(num_words=5000)  # Giới hạn 5000 từ phổ biến nhất
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)

# Chuyển đổi các chuỗi thành cùng kích thước (padding)
text_padded = pad_sequences(text_sequences, maxlen=100)

# Tạo tập dữ liệu đầu vào (X) và nhãn (y)
X = pd.concat([pd.DataFrame(authors_encoded), pd.DataFrame(category_encoded), 
               pd.DataFrame(text_padded), data[['Published', 'Updated']]], axis=1)
y = data['Target']  # Nhãn mục tiêu: Có liên quan đến "Wearable Health Monitoring" hay không (0/1)

# Chia dữ liệu thành train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bước 3: Xây dựng mô hình LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=128, input_length=100),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Dự đoán nhị phân (có liên quan hoặc không)
])

# Bước 4: Compile mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Bước 5: Huấn luyện mô hình
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Bước 6: Đánh giá mô hình
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Độ chính xác trên tập kiểm tra: {test_accuracy * 100:.2f}%')